In [1]:
%load_ext autoreload
%autoreload 2

import time
import random
import pickle, os
import numpy as np
import cvxpy as cp

from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from free_flyer.free_flyer import FreeFlyer
from free_flyer.utils import *
from solvers.mlopt_ff import MLOPT_FF
from solvers.meta import Meta

In [2]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = x_test.shape[0]

In [3]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

meta_learner = Meta(system, prob, prob_features)

n_features = 4
device_id= 1
meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

In [4]:
meta_learner.model

CNNet(
  (conv_activation): ReLU()
  (ff_activation): ReLU()
  (vars): ParameterList(
      (0): Parameter containing: [torch.cuda.FloatTensor of size 16x3x2x2 (GPU 1)]
      (1): Parameter containing: [torch.cuda.FloatTensor of size 16 (GPU 1)]
      (2): Parameter containing: [torch.cuda.FloatTensor of size 16x16x2x2 (GPU 1)]
      (3): Parameter containing: [torch.cuda.FloatTensor of size 16 (GPU 1)]
      (4): Parameter containing: [torch.cuda.FloatTensor of size 16x16x2x2 (GPU 1)]
      (5): Parameter containing: [torch.cuda.FloatTensor of size 16 (GPU 1)]
      (6): Parameter containing: [torch.cuda.FloatTensor of size 128x260 (GPU 1)]
      (7): Parameter containing: [torch.cuda.FloatTensor of size 128 (GPU 1)]
      (8): Parameter containing: [torch.cuda.FloatTensor of size 128x128 (GPU 1)]
      (9): Parameter containing: [torch.cuda.FloatTensor of size 128 (GPU 1)]
      (10): Parameter containing: [torch.cuda.FloatTensor of size 128x128 (GPU 1)]
      (11): Parameter contain

In [5]:
fn_saved = 'models/mloptff_free_flyer_20201103_1353.pt'
meta_learner.load_network(fn_saved, False)

meta_learner.model_fn

Loading presaved classifier model from models/mloptff_free_flyer_20201103_1353.pt


'/home/acauligi/cs_234/project/mlopt-micp/MetaCoCoFF_free_flyer_20201108_2056.pt'

In [7]:
meta_lsearner.train(train_data, "runs/asdfasdf")

Got to iteration 1

Got to iteration 2



RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
meta_learner.train(train_data, "runs/asdfasdf")

In [ ]:
meta_learner.finetuning(test_data, "runs/finetunning")

In [ ]:
coco_last_layer = list(meta_learner.model.parameters())[-2:]
shared_params = list(meta_learner.model.parameters())[:-2]
len(list(shared_params + coco_last_layer))

In [ ]:
# def train_meta(update_lr, meta_lr, device_id):
#     system = 'free_flyer'
#     prob_features = ['x0', 'obstacles_map']

#     meta_learner = Meta(system, prob, prob_features)

#     n_features = 4
#     meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

#     fn_saved = 'models/mloptff_free_flyer_20201103_1353.pt'
#     meta_learner.load_network(fn_saved, False)
#     print('meta_learner fn: {}'.format(meta_learner.model_fn))

#     print('Meta lr: {}'.format(meta_lr))
#     print('Update lr: {}'.format(update_lr))
#     meta_learner.update_lr = update_lr
#     meta_learner.meta_lr = meta_lr
    
#     writer_fn = 'runs/update_{}_meta_{}'.format(int(round(-100.*np.log10(meta_learner.update_lr))), int(round(-100.*np.log10(meta_learner.meta_lr))))

#     meta_learner.train(train_data, writer_fn, verbose=True)

In [ ]:
# # import multiprocessing

# # processes = []
# # for tup in tups:
# #     update_lr, meta_lr, device_id = tup
# #     p = multiprocessing.Process(target=train_meta, args=[update_lr, meta_lr, device_id])

# #     p.start()
# #     processes.append(p)
    
# # for p in processes:
# #     p.join()

# ctr = 1
# device_id = 1
# for meta_lr in [5e-4, 1e-4, 5e-5]:
#     for update_lr in [1e-5, 5e-6, 1e-6]:
#         dump_fn = './training_logs/training_{}.txt'.format(ctr)
#         file = open(dump_fn,'w')
#         file.write('On meta_lr {}\n'.format(meta_lr))
#         file.write('On update_lr {}\n'.format(update_lr))
#         file.close()
#         train_meta(update_lr, meta_lr, device_id)
#         dump_fn = './training_logs/training_{}.txt'.format(ctr)
#         file = open(dump_fn,'w')
#         file.write('On meta_lr {}\n'.format(meta_lr))
#         file.write('On update_lr {}\n'.format(update_lr))
#         file.write('Done')
#         file.close()
#         ctr += 1